In [20]:
def search(mov):
    import numpy as np
    import pandas as pd
    import warnings
    warnings.filterwarnings('ignore')
    movies=pd.read_csv('tmdb_5000_movies.csv')
    credits=pd.read_csv('tmdb_5000_credits.csv')

    movies=movies.merge(credits,on='title')

    movies=movies[['movie_id','title','release_date','overview','genres','keywords','cast','crew']]

    movies.isnull().sum()
    movies.dropna(inplace=True)
    movies.duplicated().sum()

    import ast
    def congenre(obj):
        genre=[]
        for i in ast.literal_eval(obj):
            genre.append(i['name'])
        return genre
    movies['genres']=movies['genres'].apply(congenre)
    movies['keywords']=movies['keywords'].apply(congenre)

    def concast(obj):
        L=[]
        counter=0
        for i in ast.literal_eval(obj):
            if counter!=3:
                L.append(i['name'])
                counter+=1
            else:
                break
        return L


    def Director(obj):
        L=[]
        for i in ast.literal_eval(obj):
            if i['job']=='Director':
                L.append(i['name'])
                break
        return L



    movies['crew']=movies['crew'].apply(Director)
    movies['cast']=movies['cast'].apply(concast)

    movies['overview']=movies['overview'].apply(lambda x:x.split())

    movies['cast']=movies['cast'].apply(lambda x:[i.replace(" ","") for i in x])

    movies['crew']=movies['crew'].apply(lambda x:[i.replace(" ","") for i in x])#we need to remove spaces from all the column data
    movies['keywords']=movies['keywords'].apply(lambda x:[i.replace(" ","") for i in x])
    movies['genres']=movies['genres'].apply(lambda x:[i.replace(" ","") for i in x])

    movies['tag']=movies['overview']+movies['genres']+movies['keywords']+movies['cast']+movies['crew']

    new_df=movies[['movie_id','title','tag']]
    new_df['title']=new_df['title'].apply(lambda x:x.lower())
    new_df['tag']=new_df['tag'].apply(lambda x: " ".join(x))
    new_df['tag']=new_df['tag'].apply(lambda x:x.lower())


    from sklearn.feature_extraction.text import CountVectorizer

    cv=CountVectorizer(max_features=5000,stop_words='english')

    vector_movies=cv.fit_transform(new_df['tag']).toarray()

    vector_movies.shape  # no of movies,no of words we chose


    from nltk.stem.porter import  PorterStemmer

    ps=PorterStemmer()

    def stem(text):
        y=[]
        for  i in text.split():
            y.append(ps.stem(i))

        return " ".join(y)




    new_df['tag']=new_df['tag'].apply(stem)


    vectors=cv.fit_transform(new_df['tag']).toarray()

    from sklearn.metrics.pairwise import cosine_similarity

    similarity=cosine_similarity(vectors)


    result=pd.DataFrame()

    result['movie_id']=movies['movie_id']
    result['title']=movies.title+"("+movies['release_date']+")"
    result['genre']=movies['genres'].apply(lambda x:",".join(x))
    result['cast']=movies['cast'].apply(lambda x:",".join(x))
    result['overview']=movies['overview'].apply(lambda x:" ".join(x))
    
    def recommend(mov):

            n=10 #no of movies user desire to be recommended
            movies_index=new_df[new_df['title']==mov.lower()].index[0]
            distance=similarity[movies_index]
            movies_list=sorted(list(enumerate(distance)),reverse=True,key=lambda x:x[1])[1:n+1]
            L=[]
            for i in movies_list:
                t=result.iloc[i[0]]
                df=pd.DataFrame()
                df=[t.movie_id,t.title,t.genre,t.cast,t.overview]
                L.append(df)

            header=['Movie_id','Title(Release Date)','Genres','Top 3 Cast','A Brief Overview']
            Movie=pd.DataFrame(L,columns=header)
            return Movie
    return recommend(mov)


In [21]:

UserMovie=input('Enter Movie Name: ')
Output=search(UserMovie)
Output

Enter Movie Name: avatar


,Movie_id,Title(Release Date),Genres,Top 3 Cast,A Brief Overview
0,440,Aliens vs Predator: Requiem(2007-12-25),"Fantasy,Action,ScienceFiction,Thriller,Horror","StevenPasquale,ReikoAylesworth,JohnOrtiz","A sequel to 2004's Alien vs. Predator, the ico..."
1,679,Aliens(1986-07-18),"Horror,Action,Thriller,ScienceFiction","SigourneyWeaver,MichaelBiehn,JamesRemar",When Ripley's lifepod is found by a salvage cr...
2,270938,Falcon Rising(2014-09-05),"Adventure,Action","MichaelJaiWhite,NealMcDonough,LateefCrowder","Chapman is an ex-marine in Brazil's slums, bat..."
3,602,Independence Day(1996-06-25),"Action,Adventure,ScienceFiction","WillSmith,BillPullman,JeffGoldblum","On July 2, a giant alien mothership enters orb..."
4,7450,Titan A.E.(2000-06-16),"Animation,Action,ScienceFiction,Family,Adventure","MattDamon,BillPullman,DrewBarrymore",A young man finds out that he holds the key to...
5,44943,Battle: Los Angeles(2011-03-08),"Action,ScienceFiction","AaronEckhart,RamónRodríguez,WillRothhaar",The Earth is attacked by unknown forces. As pe...
6,34851,Predators(2010-07-03),"Action,ScienceFiction,Adventure,Thriller","TopherGrace,AliceBraga,LaurenceFishburne",A mercenary reluctantly leads a motley crew of...
7,11551,Small Soldiers(1998-07-10),"Comedy,Adventure,Fantasy,ScienceFiction,Action","GregorySmith,KirstenDunst,DenisLeary",When missile technology is used to enhance toy...
8,76757,Jupiter Ascending(2015-02-04),"ScienceFiction,Fantasy,Action,Adventure","MilaKunis,ChanningTatum,SeanBean",In a universe where human genetic material is ...
9,11260,Meet Dave(2008-07-08),"Comedy,ScienceFiction,Adventure,Family","EddieMurphy,ElizabethBanks,GabrielleUnion",A crew of miniature aliens operate a spaceship...
